## Validation of Annual Summer Irrigated Cropping Area (SICA)
Using a variety of measures. Firstly, compare the overall area between the validation dataset and the prediction dataset, secondly calculate a Jaccard Similarity Score to test how similar the two datasets are, then generate confusion matrix with a number of measures describing the precion, omission, commission, accuracy etc.

Relying on Claire Kraus's notebook for guidance:

https://github.com/GeoscienceAustralia/dea-notebooks/blob/ClaireK/Crop_mapping/NamoiPilotProjectWorkflow/ValidateAutomaticIrrigatedCropAreaGeotiffs.ipynb

In [ ]:
import numpy as np
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt

#import custom functions
import sys
sys.path.append('src')
import DEAPlotting, SpatialTools
from SpatialTools import geotransform

### User Inputs

In [ ]:
#provide the filepaths to the irrigated cropping extent tif and the validation shapefile
irrigated = "/g/data/r78/cb3058/dea-notebooks/ICE_project/data/NSW_mdb_mask.tif" 

# validation = "/g/data/r78/cb3058/dea-notebooks/ICE_project/data/spatial/nmdb_OEH2017_irrigated.shp"
validation = "/g/data1a/r78/cb3058/dea-notebooks/ICE_project/data/nmdb_OEH2017_irrigated.tif"
# clip_shp = "/g/data/r78/cb3058/dea-notebooks/ICE_project/data/spatial/nmdb_individual_catchments/NAMOI RIVER.shp"

#what year are we validating
year = '1999-00'

### Prepare the datasets

In [ ]:
#open the irrigatation tif
irr  = xr.open_rasterio(irrigated).drop('band').squeeze()
#grab some transform info from it
# transform, projection = transform_tuple(irr, (irr.x, irr.y), epsg=3577)
width,height = irr.shape
#rasterize the catchment boundaries that encompass our validation area
# boundary = SpatialTools.rasterize_vector(clip_shp, height, width,
#                                          transform, projection, raster_path=None)
#clip extent to the catchment boundaries
# irr = irr.where(boundary)
#count number of nans
# num_of_nans_pred = np.isnan(irr.values).sum()
#convert to a boolean array of irr/not-irr
AutomaticCropBoolean  = irr.values

In [ ]:
ValidationMaskBoolean = xr.open_rasterio(validation).drop('band').squeeze()
ValidationMaskBoolean = ValidationMaskBoolean.values.astype(bool)

In [ ]:
#convert validation shapefile to array first
transform, projection = transform_tuple(irr, (irr.x, irr.y), epsg=3577)
width,height = irr.shape

ValidationMaskBoolean  = SpatialTools.rasterize_vector(validation, height, width,
                                            transform, projection, raster_path=None)

# ValidationMaskBoolean = np.where(boundary, ValidationMaskBoolean, 0
ValidationMaskBoolean = ValidationMaskBoolean.astype(bool)

In [ ]:
SpatialTools.array_to_geotiff("/g/data1a/r78/cb3058/dea-notebooks/ICE_project/data/nmdb_OEH2017_irrigated.tif",
          ValidationMaskBoolean,
          geo_transform = transform, 
          projection = projection, 
          nodata_val = 0)

In [ ]:
# plt.figure(figsize=(10,10))
# plt.imshow(ValidationMaskBoolean)

### Non-site specific accuracy (compare areas)

In [ ]:
validation_area = np.count_nonzero(ValidationMaskBoolean) *(25*25) / 10000
irrigated_area = np.count_nonzero(AutomaticCropBoolean)*(25*25) / 10000
print("The area under irrigation in the validation dataset is: " + str(validation_area) + " ha")
print("The area under irrigation in the irrigated area dataset is: " + str(irrigated_area) + " ha")
print("irrigated vs validation % is : " + str(round((irrigated_area/validation_area*100), 1)))

### Jaccard Similarity index

In [ ]:
from sklearn.metrics import accuracy_score, jaccard_similarity_score

In [ ]:
jss = jaccard_similarity_score(ValidationMaskBoolean, AutomaticCropBoolean, normalize=True)
ac = accuracy_score(ValidationMaskBoolean, AutomaticCropBoolean, normalize=True)

In [ ]:
print("The Normalised Jaccard Similarity Score is: "+ str(round(jss, 2)))
print("The Normalised Accuracy Score is: "+ str(round(ac, 2)))

### Confusion matrix

In [ ]:
YesRealYesAuto = np.logical_and(AutomaticCropBoolean, ValidationMaskBoolean)
NoRealNoAuto = np.logical_and(~AutomaticCropBoolean, ~ValidationMaskBoolean)

YesRealNoAuto = np.logical_and(AutomaticCropBoolean, ~ValidationMaskBoolean)
NoRealYesAuto = np.logical_and(~AutomaticCropBoolean, ValidationMaskBoolean)

In [ ]:
Correct_positives = YesRealYesAuto.sum()
Incorrect_positives = NoRealYesAuto.sum()
Correct_negatives = NoRealNoAuto.sum()
Incorrect_negatives = YesRealNoAuto.sum()

Totalpixels = (width * height)

Accuracy = (Correct_positives + Correct_negatives) / Totalpixels
Misclassification_rate = (Incorrect_positives + Incorrect_negatives) / Totalpixels
True_Positive_Rate = Correct_positives / ValidationMaskBoolean.sum()
False_Positive_Rate = Correct_positives / ((~ValidationMaskBoolean).sum())
Specificity = Correct_negatives / ((~ValidationMaskBoolean).sum())
Precision = Correct_positives / AutomaticCropBoolean.sum()
Prevalence = (ValidationMaskBoolean.sum() ) / Totalpixels

In [ ]:
print('\033[1m' + '{0} Automatic Irrigated Crop Extent'.format(year) + '\033[0m')
print('Accuracy = %.5f' % Accuracy)
print('Misclassification_rate = %.5f' % Misclassification_rate)
print('True_Positive_Rate = %.5f' % True_Positive_Rate)
print('False_Positive_Rate = %.5f' % False_Positive_Rate)
print('Specificity = %.5f' % Specificity)
print('Precision = %.5f' % Precision)
print('Prevalence = %.5f' % Prevalence)
print("#################")
print("The area of incorrect positives is" + str(Incorrect_positives * (25*25)/10000))
print("The area of correct positives is" + str(Correct_positives * (25*25)/10000))
print("The area of incorrect negatives is" + str(Incorrect_negatives * (25*25)/10000))